# sumaryzacja tekstu

In [1]:
!pip install "datasets<3.0.0"


## importowanie datesetu

In [2]:
from datasets import load_dataset


ds = load_dataset("xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 750)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [3]:
print(ds)
print(shots)

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 32053
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 1727
    })
})
Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2
})


In [4]:
!pip install bert-extractive-summarizer

In [5]:
from summarizer import Summarizer
model = Summarizer()

In [6]:
!pip install pytorch-ignite
!pip install torchmetrics

In [7]:
from ignite.metrics import Rouge

m = Rouge(variants=["L", 2], multiref="best")

candidate = "the cat is not there".split()
references = [
    "the cat is on the mat".split(),
    "there is a cat on the mat".split()
]

m.update(([candidate], [references]))

print(m.compute())

{'Rouge-L-P': 0.6, 'Rouge-L-R': 0.5, 'Rouge-L-F': 0.5, 'Rouge-2-P': 0.5, 'Rouge-2-R': 0.4, 'Rouge-2-F': 0.4}


In [8]:
from pprint import pprint
from torchmetrics.text.bert import BERTScore
preds = """
The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we're neglected or forgotten," she said.
"That may not be true but it is perhaps my perspective over the last few days.
"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"
Meanwhile, a flood alert remains in place across the Borders because of the constant rain.
Peebles was badly hit by problems, sparking calls to introduce more defences in the area.
Scottish Borders Council has put a list on its website of the roads worst affected and drivers have been urged not to ignore closure signs.
The Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand.
He said it was important to get the flood protection plan right but backed calls to speed up the process.
"I was quite taken aback by the amount of damage that has been done," he said.
"Obviously it is heart-breaking for people who have been forced out of their homes and the impact on businesses."
He said it was important that "immediate steps" were taken to protect the areas most vulnerable and a clear timetable put in place for flood prevention plans.
Have you been affected by flooding in Dumfries and Galloway or the Borders? Tell us about your experience of the situation and how it was handled. Email us on selkirk.news@bbc.co.uk or dumfries@bbc.co.uk.
"""
result = model(preds)
# print(result)
target ="""
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.
"""
print(f'target: {target}')
print(f'result: {result}')
bertscore = BERTScore(model_name_or_path="microsoft/deberta-v3-large", lang='en')
pprint(bertscore([result], [target]))

target: 
Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.

result: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. First Minister Nicola Sturgeon visited the area to inspect the damage. "That may not be true but it is perhaps my perspective over the last few days. He said it was important to get the flood protection plan right but backed calls to speed up the process.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

{'f1': tensor(0.7959), 'precision': tensor(0.7827), 'recall': tensor(0.8095)}


In [9]:
num_workers = 1
batch_size = 1

In [10]:
from torch.utils.data import DataLoader
import torch
test_dataset = ds["test"]
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
print(test_dataset)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 1727
})


In [48]:
def bert_score_loss(batch, model, bert: bool, tokenizer: any, prompt = None):
    if bert:
        predictions = [model(doc, ratio=0.1) for doc in batch['document']]
    else:
        docs = [doc for doc in batch['document']]
        prompted  = [f'{prompt}{doc}'for doc in batch['document']]
        inputs = tokenizer(
        prompted,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=4096
    )
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=100,
        do_sample=True,
        top_k=0,
        temperature=0.6,
    )
        input_length = inputs["input_ids"].shape[1]
        new_tokens = summary_ids[:, input_length:]
        predictions = tokenizer.batch_decode(
            new_tokens, skip_special_tokens=True
        )
    print(f"Doc: {batch['document'][0][:100]}")
    print(f"Prediction: {predictions[0]}")
    print(f"Target: {batch['summary'][0]}")
    predictions = [pred if pred.strip() else "." for pred in predictions]
    return bertscore(predictions, batch["summary"])

In [12]:
def batch_to_device(batch: dict, device) -> dict:
    return {k: v.to(device) for k, v in batch.items()}

In [13]:
import torch
import torch.nn as nn
from torch import Tensor
def calc_loss_batch(batch: dict, model: nn.Module) -> Tensor:
    logits = model(batch['input_ids'])
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), batch['labels'].flatten())
    return loss

In [14]:
def calc_loss_loader(data_loader, model, device, num_batches: int, bert: bool, tokenizer: any,prompt = None) -> float:
    total_loss = {'precision': [], 'recall': [], 'f1': []}
    for i, batch in enumerate(data_loader):
        if i >= num_batches:
            break
        # batch = batch_to_device(batch, device)
        loss = bert_score_loss(batch, model, bert, tokenizer, prompt)
        total_loss['precision'].append(loss['precision'].unsqueeze(0))
        total_loss['recall'].append(loss['recall'].unsqueeze(0))
        total_loss['f1'].append(loss['f1'].unsqueeze(0))
    total_loss['precision'] = torch.cat(total_loss['precision'])
    total_loss['recall'] = torch.cat(total_loss['recall'])
    total_loss['f1'] = torch.cat(total_loss['f1'])
    print(total_loss)
    return (sum(total_loss['precision'])/ len(total_loss['precision']),sum(total_loss['recall'])/ len(total_loss['recall']),sum(total_loss['f1'])/ len(total_loss['f1']))

In [15]:
def evaluate_model(model: nn.Module, loader: DataLoader,
                    device, eval_iter: int, bert: bool, tokenizer: any, prompt="") -> tuple[float, float]:
    if not bert:
      model.eval()
    with torch.no_grad():
        loss = calc_loss_loader(loader, model, device, num_batches=eval_iter, bert=bert, tokenizer=tokenizer, prompt=prompt)
    return loss

In [ ]:
from torch import nn
loss = evaluate_model(model, test_loader, device='cuda', eval_iter=2, bert=True, tokenizer=None)
print(loss)



Doc: The massive crater is filled with debris, household items and vehicles.
The hole is just four miles 
Prediction: The massive crater is filled with debris, household items and vehicles. The hole is just four miles from the Arena das Dunas stadium, which is set to host the Group D match between Italy and Uruguay on Tuesday.
Target: Houses in a favela, near one of Brazil's World Cup stadiums, have been evacuated after this huge sinkhole opened up.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

Doc: Police said that someone opened fire from a passing car and witnesses spoke of up to a dozen shots b
Prediction: Police said that someone opened fire from a passing car and witnesses spoke of up to a dozen shots being fired close to the main police station. The shots were fired in Friedrich-Stoltze Square, a busy area of bars and cafes, as local people enjoyed the Ascension Day holiday in the sunshine.
Target: At least three people were wounded, two seriously, when shots were fired in the German city of Frankfurt on Thursday.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

Doc: The car tried to crash through the barriers at Place de la Concorde, and police opened fire as it sp
Prediction: The car tried to crash through the barriers at Place de la Concorde, and police opened fire as it sped away. A third person, a young woman in hospital with bullet wounds, is also thought to have been in the car.
Target: Paris officials, investigating why a car was driven through the security barriers ahead of the Tour de France on Sunday morning, are holding two people who handed themselves in overnight.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

Doc: Media playback is unsupported on your device
8 May 2015 Last updated at 10:28 BST
During the war, fa
Prediction: Media playback is unsupported on your device
8 May 2015 Last updated at 10:28 BST
During the war, families would have to ration their food and had little communication in their homes. Luxuries like chocolate and fruit were very difficult to find and families had to grow their own food to survive.
Target: Martin went to the German Occupation Museum to see what life was like for a family living on Guernsey in World War II.


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, enc

KeyboardInterrupt: 

In [17]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.padding_side = 'left'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer.pad_token_id = tokenizer.eos_token_id
model2 = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

The following layers were not sharded: transformer.h.*.mlp.c_fc.weight, transformer.ln_f.bias, lm_head.weight, transformer.h.*.mlp.c_proj.weight, transformer.h.*.mlp.c_proj.bias, transformer.h.*.attn.c_attn.bias, transformer.wte.weight, transformer.wpe.weight, transformer.ln_f.weight, transformer.h.*.ln_*.weight, transformer.h.*.attn.c_attn.weight, transformer.h.*.attn.c_proj.weight, transformer.h.*.mlp.c_fc.bias, transformer.h.*.ln_*.bias, transformer.h.*.attn.c_proj.bias


In [44]:
def generate_prompt(tokenizer, size):
    prompt = ""
    shots_loader = DataLoader(shots, batch_size=100000, shuffle=True, num_workers=num_workers)
    for batch in shots_loader:
        for i in range(len(batch['document'])):
            prompt += "Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? "
            prompt += f'{i+1}. {batch['document'][i]} -> {batch['summary'][i]}'
            prompt += tokenizer.eos_token

    prompt += "Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about?:\n"
    return prompt

prompt = generate_prompt(tokenizer, 3)
print(prompt)

Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? 1. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez levelled from close range.
World champions Germany moved above Chile with a 3-1 win over Cameroon and will play Mexico on Thursday (19:00).
RB Leipzig forward Timo Werner scored twice, with Hoffenheim midfielder Kerem Demirbay also on target for the experimental German squad competing in Russia.
Porto forward Vincent Aboubakar netted for Cameroon, who finished bottom of the group, with a draw from their match against Australia their only point.
Group A:
Group B: -> Chile qualified behind Germany from Group B to reach the Confederations Cup semi-finals after drawing 1-1 with Australia in their final group game.<|endoftext|>Summarize this text. Don't hallucinate. Be thorough and explicit. Always prov

In [49]:
print(prompt)
print(evaluate_model(model2, test_loader, device='cuda', eval_iter=10, bert=False, tokenizer=tokenizer, prompt = prompt))


Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? 1. Euro 2016 winners Portugal will now face Chile in the last four on Wednesday (19:00 BST).
Australia went ahead through winger James Troisi, before substitute Martin Rodriguez levelled from close range.
World champions Germany moved above Chile with a 3-1 win over Cameroon and will play Mexico on Thursday (19:00).
RB Leipzig forward Timo Werner scored twice, with Hoffenheim midfielder Kerem Demirbay also on target for the experimental German squad competing in Russia.
Porto forward Vincent Aboubakar netted for Cameroon, who finished bottom of the group, with a draw from their match against Australia their only point.
Group A:
Group B: -> Chile qualified behind Germany from Group B to reach the Confederations Cup semi-finals after drawing 1-1 with Australia in their final group game.<|endoftext|>Summarize this text. Don't hallucinate. Be thorough and explicit. Always prov

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: The emergency services were called to Glenapp Road in the Pollokshields area of the city at about 11
Prediction: 
The initial assessment was that one of the victims was seriously ill.
One resident who was taking chloramphenicol was admitted and continued taking chloramphenicol.
Both of the victims have been released from hospital. The other did not have any symptoms.
The emergency services are looking for two persons.
The health department is at the scene and is assessing the circumstances of their death.
The man in the blue and black coat, who lives in the flat at the time of the
Target: A fire in a block of tenement flats in Glasgow has left two people requiring hospital treatment.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: Media playback is not supported on this device
The League One side were ahead after just three minut
Prediction: 
Media playback is not supported on this device Stuckmann calls for him to be given a chance to score
Media playback is not supported on this device Stuckmann praises Stuckmann's work
Media playback is not supported on this device Stuckmann: 'We have seen things'
Media playback is not supported on this device Stuckmann: 'No one knows what's going on but I'm excited to get out for the game'
Crowley was at his best when
Target: Andy Williams' double saw Doncaster ease to a comfortable victory over National League North side Stalybridge.


...

Doc: Clubs are required to provide accurate details of training sessions and player whereabouts so they a
Prediction: 
A statement from the club's official website said: "The club has made a significant effort to ensure that all information is provided to the correct and complete accounts.
"We have also made a number of changes to our policy and procedures to ensure that any information we provide to the correct and complete accounts is provided to our players and coaches as soon as possible before they are asked to take part in a training session.
"This has been in place for over a year and all information that was
Target: Bournemouth have accepted a Football Association charge relating to a breach of anti-doping rules.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: A masked man holding a gun approached him at about 10:35 BST in a car park in Glenarm on Sunday and 
Prediction:  – Stuff
Target: A man has been robbed at gunpoint as he was taking photographs in a village on the County Antrim coast.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias


{'precision': tensor([0.6992, 0.6541, 0.7629, 0.3336, 0.6442, 0.7159, 0.7045, 0.7350, 0.7361,
        0.6095]), 'recall': tensor([0.7554, 0.7262, 0.7543, 0.0952, 0.7461, 0.7374, 0.7564, 0.7985, 0.7717,
        0.5506]), 'f1': tensor([0.7262, 0.6883, 0.7586, 0.1482, 0.6914, 0.7265, 0.7296, 0.7654, 0.7535,
        0.5785])}
(tensor(0.6595), tensor(0.6692), tensor(0.6566))


In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer3 = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B")
tokenizer3.padding_side = 'left'
if tokenizer3.pad_token is None:
    tokenizer3.pad_token = tokenizer3.eos_token
    # Some models also require explicitly setting the pad_token_id
    tokenizer3.pad_token_id = tokenizer3.eos_token_id
model3 = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B")




The following TP rules were not applied on any of the layers: {'layers.*.self_attn.q_proj': 'colwise', 'layers.*.self_attn.k_proj': 'colwise', 'layers.*.self_attn.v_proj': 'colwise', 'layers.*.self_attn.o_proj': 'rowwise', 'layers.*.mlp.gate_proj': 'colwise', 'layers.*.mlp.up_proj': 'colwise', 'layers.*.mlp.down_proj': 'rowwise'}
The following layers were not sharded: lm_head.weight, model.layers.*.post_attention_layernorm.weight, model.embed_tokens.weight, model.layers.*.input_layernorm.weight, model.norm.weight, model.layers.*.self_attn.q_proj.bias, model.layers.*.self_attn.k_proj.weight, model.layers.*.self_attn.v_proj.weight


In [50]:
prompt3 = generate_prompt(tokenizer3, 0)
print(evaluate_model(model3, test_loader, device='cuda', eval_iter=10, bert=False, tokenizer=tokenizer3, prompt=prompt3))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: Guinea finish bottom of Group A having needed a win against Argentina to have any chance of progress
Prediction: -> A group of African countries went through to the Confederations Cup semi-finals after losing to Argentina, but failed to advance to the last 16 due to a draw against South Korea. Guinea was the only African team to make it through to the last 16.

Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? 1. "The 2018 FIFA World Cup, commonly known as the
Target: Argentina thrashed Guinea 5-0 on Friday at the Under-20 World Cup in South Korea to knock the African side out of the tournament.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.

Doc: The county council plans to build 200 council homes and 300 affordable homes.
The first will be buil
Prediction:  This will include the building of a new council house and a new school.
A new school, which will be built on the site of the former local authority building, will be opened by the council in September.
A proposal for a new housing development on the site of the former Custom House Lane, in Connah's Quay, has been submitted to the council’s planning committee. The site is located on the northern edge of Dumfries, close to the town centre. The first phase of the
Target: Building work is due to start next month on the first new council homes in Flintshire for more than 20 years.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.

...

Doc: Straker, 27, has been without a club since leaving York in January, where he made 27 appearances.
He
Prediction:  Grimsby boss Steve Bruce insists his team have not lost too many key players.
He said: "We have had a lot of changes to the squad over the last few weeks, but we are building a strong squad with a few of our best players."
The 20-year-old said: "We have had a summer that has been great for us, but we have had a few changes to the squad. We have had a few changes to the squad over the last few weeks and I
Target: Grimsby Town have signed former York City midfielder Anthony Straker on a deal until the end of the season.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.

Doc: After 25 years, the Royal National Mission to Deep Sea Fishermen closed its doors in Eyemouth in 201
Prediction:  And all the information should be available for people to sign up to the mailing list for updates. The Royal National Mission to Deep Sea Fishermen, which was founded in 1975, is a charity that works to protect and conserve deep sea fish stocks.
The mission is now closed, and the building is now being used as a community centre. The building was built in the late 18th century and has had many extensions over the years. The building was once used as a gran
Target: A former fishermen's mission in the Scottish Borders is being converted into an arts and performance centre.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias


{'precision': tensor([0.7198, 0.7601, 0.4966, 0.7010, 0.7142, 0.6851, 0.7331, 0.7660, 0.7123,
        0.7815]), 'recall': tensor([0.7935, 0.7890, 0.6224, 0.7713, 0.7794, 0.7573, 0.8120, 0.8138, 0.7650,
        0.8251]), 'f1': tensor([0.7549, 0.7743, 0.5524, 0.7345, 0.7454, 0.7194, 0.7706, 0.7892, 0.7377,
        0.8027])}
(tensor(0.7070), tensor(0.7729), tensor(0.7381))


In [42]:
prompt4 = "Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer What is this text about? "
print(
    evaluate_model(
        model3,
        test_loader,
        device="cuda",
        eval_iter=10,
        bert=False,
        tokenizer=tokenizer3,
        prompt=prompt4,
    )
)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Doc: 7 July 2016 Last updated at 09:49 BST
Speaking to Today programme presenter John Humphrys, he accept
Prediction:  "I don't think it's fair to say that I was against the war. If I'd been in the position of the leadership of the British government, I'd have been opposed to anything that was going on. I would not have been able
Target: Tony Blair says people will not accept that he means his regret over mistakes in the Iraq war until he disowns the decision to join the US coalition to topple Saddam Hussein.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.

Doc: PC Keith Burgess, 41, who was based at Southampton Central police station, also faces three counts o
Prediction: 
Target: A Hampshire police officer has been charged with making indecent images of a child, sexual assault and grooming a girl under 16.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.

...

Doc: Bissouma penned a professional deal with the northern club only in July 2016, four months after join
Prediction:  He is 21 years old, and is expected to make his debut for Lille in the season finale against Lyon on 28th May.
Bissouma has 10 goals in 20 appearances in Ligue 1
Target: Mali international midfielder Yves Bissouma has signed a new contract to stay with French club Lille until 2021.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.

Doc: The 29-year-old, who carded a round of 59 in January, began the final round with a four-stroke lead.
Prediction: 
Target: Canadian Adam Hadwin survived a late scare to claim his maiden PGA title with a one-shot win at the Valspar Championship in Innisbrook, Florida.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias


{'precision': tensor([0.7634, 0.2818, 0.7821, 0.3027, 0.7753, 0.2613, 0.7755, 0.7896, 0.7934,
        0.2367]), 'recall': tensor([0.7813, 0.0942, 0.8012, 0.0776, 0.7891, 0.0968, 0.8196, 0.8093, 0.8428,
        0.0645]), 'f1': tensor([0.7722, 0.1412, 0.7915, 0.1235, 0.7821, 0.1413, 0.7970, 0.7993, 0.8173,
        0.1014])}
(tensor(0.5762), tensor(0.5176), tensor(0.5267))


In [41]:
print(
    evaluate_model(
        model2,
        test_loader,
        device="cuda",
        eval_iter=2,
        bert=False,
        tokenizer=tokenizer,
        prompt=prompt4,
    )
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: United took the lead on the night and in the tie when Henrikh Mkhitaryan drilled in a low finish in 
Prediction: 
The draw will be played on Saturday before the match against Ajax at home on Saturday.
United manager Jose Mourinho has made it clear he wants to play a big game against a team that was not there on Sunday.
"I don't want
Target: Marcus Rashford's extra-time goal sent Manchester United into the Europa League semi-final beating Anderlecht 2-1 on a nervy night at Old Trafford.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Doc: 20 May 2016 Last updated at 12:40 BST
They adopted him and he came into the studio for a live interv
Prediction: 
The first part is about being a "robot", and being a robot. In the first part, he appears to be a man, dressed in a suit, with a white shirt and a turban. He looks like he's in a
Target: Stray dog Arthur crossed the Amazon river in South America to follow Swedish explorers who befriended him.


The following layers were not sharded: encoder.layer.*.output.dense.weight, encoder.layer.*.attention.self.key_proj.bias, encoder.layer.*.attention.self.value_proj.bias, encoder.layer.*.attention.output.dense.weight, encoder.layer.*.attention.self.query_proj.bias, encoder.layer.*.attention.self.value_proj.weight, encoder.layer.*.attention.output.dense.bias, encoder.LayerNorm.weight, embeddings.LayerNorm.weight, encoder.layer.*.intermediate.dense.weight, encoder.layer.*.output.LayerNorm.weight, encoder.layer.*.output.dense.bias, encoder.LayerNorm.bias, encoder.layer.*.attention.self.key_proj.weight, encoder.layer.*.attention.output.LayerNorm.weight, encoder.rel_embeddings.weight, encoder.layer.*.attention.self.query_proj.weight, encoder.layer.*.intermediate.dense.bias, embeddings.word_embeddings.weight, encoder.layer.*.attention.output.LayerNorm.bias, encoder.layer.*.output.LayerNorm.bias, embeddings.LayerNorm.bias


{'precision': tensor([0.7578, 0.7555]), 'recall': tensor([0.7615, 0.7870]), 'f1': tensor([0.7597, 0.7709])}
(tensor(0.7567), tensor(0.7743), tensor(0.7653))
